# Examples for QMSim.jl

This demonstrates the basic functionality of `QMSim.jl` from matrix construction all the way to solving real quantum mechanics problems.

In [1]:
# activate the examples environment
import Pkg;
Pkg.activate(@__DIR__)
# Pkg.add("Term")
# Pkg.resolve()
# Pkg.instantiate()
# Pkg.precompile()
# Pkg.update()

  Activating project at `~/Julia/QMSim.jl/examples`


In [3]:
ENV["COLUMNS"] = 160

using Revise
using SparseArrays, LinearAlgebra

using QGas.NumericalTools.ArrayDimensions: Dimensions

using QMSim

## Demonstrate the creation of some simple rules

In [7]:
    function ham_tunneling(x, y; J=1.0)
        return -J
    end

    RelativeRule(ham_tunneling, [1,])
    ExplicitRule( Diagonal(collect(1:5) .+ 0.0im) )

ExplicitRule{Diagonal{ComplexF64, Vector{ComplexF64}}}(ComplexF64[1.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 2.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … 4.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 5.0 + 0.0im])

## Demonstrate the overall setup of a matrix

This does not solve any problems.

In [ ]:
# create an array of physical dimensions for our quantum system to live in, including an example with two space and one spin dimensions

dims = Dimensions(
    DimensionWithSpace(; x0=-10.0, dx=1.0, npnts=5, unit="X Momentum", periodic=true, spatial=true),
    # DimensionWithSpace(; x0=-1.0, dx=1.0, npnts=3, unit="Spin", periodic=true, spatial=true),
)

"""
    build_rules!(mwr::MatrixWithRules)

add required rules to the matrix
"""
function build_rules!(mwr::MatrixWithRules)

    function ham_tunneling(x, y; J=1.0)
        return -J
    end

    add_rule!(mwr, RelativeRule, ham_tunneling, [1,])
    add_rule!(mwr, RelativeRule, ham_tunneling, [-1,])
    add_rule!(mwr, ExplicitRule, Diagonal(collect(1:5) .+ 0.0im))
    return mwr
end

# See how we define the type of the matrix we want!
mwr = MatrixWithRules(SparseMatrixCSC{ComplexF64}, dims)

build_rules!(mwr)

generate_builders!(mwr)

build!(mwr; J=2.0)

mwr

5×5 MatrixWithRules{ComplexF64, SparseMatrixCSC{ComplexF64}}:
  1.0+0.0im  -2.0+0.0im   0.0+0.0im   0.0+0.0im  -2.0+0.0im
 -2.0+0.0im   2.0+0.0im  -2.0+0.0im   0.0+0.0im   0.0+0.0im
  0.0+0.0im  -2.0+0.0im   3.0+0.0im  -2.0+0.0im   0.0+0.0im
  0.0+0.0im   0.0+0.0im  -2.0+0.0im   4.0+0.0im  -2.0+0.0im
 -2.0+0.0im   0.0+0.0im   0.0+0.0im  -2.0+0.0im   5.0+0.0im

Now we do the same thing, but for more than one matrix:

In [4]:
mwrs = MatricesWithRules(SparseMatrixCSC{ComplexF64}, dims)

add_matrix!(mwrs, :tunneling)
add_matrix!(mwrs, :potential)

function ham_tunneling(x, y; J=1.0)
    return -J
end

add_rule!(mwrs, :tunneling, RelativeRule, ham_tunneling, [1,])
add_rule!(mwrs, :tunneling, RelativeRule, ham_tunneling, [-1,])
add_rule!(mwrs, :potential, ExplicitRule, Diagonal(collect(1:5) .+ 0.0im))

generate_builders!(mwrs)

build!(mwrs; J=2.0)

mwrs.matrix

5×5 SparseMatrixCSC{ComplexF64, Int64} with 15 stored entries:
  1.0+0.0im  -2.0+0.0im       ⋅           ⋅      -2.0+0.0im
 -2.0+0.0im   2.0+0.0im  -2.0+0.0im       ⋅           ⋅    
      ⋅      -2.0+0.0im   3.0+0.0im  -2.0+0.0im       ⋅    
      ⋅           ⋅      -2.0+0.0im   4.0+0.0im  -2.0+0.0im
 -2.0+0.0im       ⋅           ⋅      -2.0+0.0im   5.0+0.0im

In [12]:
mwrs = MatricesWithRules(SparseMatrixCSC{ComplexF64}, dims)

add_matrix!(mwrs, :tunneling)
add_matrix!(mwrs, :potential)

function ham_tunneling(x, y; J=1.0)
    return -J
end

add_rule!(mwrs, :tunneling, RelativeRule, ham_tunneling, [1,])
add_rule!(mwrs, :tunneling, RelativeRule, ham_tunneling, [-1,])
add_rule!(mwrs, :potential, ExplicitRule, Diagonal(collect(1:5) .+ 0.0im))

set_default_kwargs!(mwrs, :tunneling; J=4.0)

generate_builders!(mwrs)

build!(mwrs; J=2.0)

mwrs.matrix

5×5 SparseMatrixCSC{ComplexF64, Int64} with 15 stored entries:
  1.0+0.0im  -2.0+0.0im       ⋅           ⋅      -2.0+0.0im
 -2.0+0.0im   2.0+0.0im  -2.0+0.0im       ⋅           ⋅    
      ⋅      -2.0+0.0im   3.0+0.0im  -2.0+0.0im       ⋅    
      ⋅           ⋅      -2.0+0.0im   4.0+0.0im  -2.0+0.0im
 -2.0+0.0im       ⋅           ⋅      -2.0+0.0im   5.0+0.0im

At this point this is not all that useful, but the main point is that with slightly more syntactic sugar we can write code to define pretty much any physics problem!

## Solve specific problems

The existing code already encapsulates any abstract matrix.  There is some work required to make this work efficiently for both dense and sparse matrices.

Here is an example for a dense matrix including performance profiling

In [ ]:
dims = Dimensions(
    DimensionWithSpace(; x0=-10.0, dx=1.0, npnts=5001, unit="X Momentum", periodic=true, spatial=true),
)

qms = QMSolver(Matrix{ComplexF64}, dims; num_states=6, wrap=Hermitian)

add_matrix!(qms, :tunneling)
add_matrix!(qms, :potential)
add_rule!(qms, :potential, ExplicitRule, Diagonal(4 .* ones(ComplexF64, 5001)))

function ham_tunneling(x, y; J=1.0)
    return -J
end

add_rule!(qms, :tunneling, RelativeRule, ham_tunneling, [1,])
add_rule!(qms, :tunneling, RelativeRule, ham_tunneling, [-1,])

generate_builders!(qms)

eigensystem!(qms; J=2.0)
qms.eigenvalues

5001-element Vector{ComplexF64}:
 -4.4064830822505436e-16 + 0.0im
    3.157010062427297e-6 + 0.0im
   3.1570100632255443e-6 + 0.0im
   1.2628035266754423e-5 + 0.0im
   1.2628035267677318e-5 + 0.0im
    2.841306066284507e-5 + 0.0im
   2.8413060663925683e-5 + 0.0im
    5.051206133419378e-5 + 0.0im
   5.0512061335232256e-5 + 0.0im
    7.892500239699925e-5 + 0.0im
                         ⋮
       7.999936070708113 + 0.0im
       7.999961326683964 + 0.0im
       7.999961326683964 + 0.0im
       7.999980268700735 + 0.0im
       7.999980268700735 + 0.0im
       7.999992896728527 + 0.0im
       7.999992896728527 + 0.0im
       7.999999210747406 + 0.0im
       7.999999210747406 + 0.0im

In [ ]:
dims = Dimensions(
    DimensionWithSpace(; x0=-10.0, dx=1.0, npnts=5001, unit="X Momentum", periodic=true, spatial=true),
)

qms = QMSolver(SparseMatrixCSC{ComplexF64}, dims; num_states=50, wrap=Hermitian)

add_matrix!(qms, :tunneling)
add_matrix!(qms, :potential)

function ham_tunneling(x, y; J=1.0)
    return -J
end

add_rule!(qms, :tunneling, RelativeRule, ham_tunneling, [1,])
add_rule!(qms, :tunneling, RelativeRule, ham_tunneling, [-1,])
add_rule!(qms, :potential, ExplicitRule, Diagonal(4 .* ones(ComplexF64, 5001)))

generate_builders!(qms)

eigensystem!(qms; J=2.0)
qms.eigenvalues

50-element Vector{ComplexF64}:
 1.5248184832292898e-16 + 0.0im
   3.157010062542175e-6 + 0.0im
   3.157010062709666e-6 + 0.0im
  1.2628035266956139e-5 + 0.0im
  1.2628035266982345e-5 + 0.0im
   2.841306066309335e-5 + 0.0im
  2.8413060663251837e-5 + 0.0im
   5.051206133433723e-5 + 0.0im
   5.051206133484585e-5 + 0.0im
   7.892500239755684e-5 + 0.0im
                        ⋮
  0.0013921608585866138 + 0.0im
  0.0013921608585869382 + 0.0im
  0.0015278957919843872 + 0.0im
  0.0015278957919847322 + 0.0im
  0.0016699423337169562 + 0.0im
  0.0016699423337193228 + 0.0im
  0.0018183002595615729 + 0.0im
  0.0018183002595625866 + 0.0im
   0.001972969335335496 + 0.0im

In [ ]:
## Demonstrate how to create a new type of QMSolver for a specific problem

module Npod

using QMSim

"""
    NpodSolver <: AbstractMatrixSolver

A subtype of `AbstractMatrixSolver` that setups and solves the N-pod problem.  This describes two-level atom, a lambda-scheme, a tripod, and anything with one level coupled to N-sub-levels. 
"""
struct NpodSolver{T, AbstractMatrix{T}} <: AbstractMatrixSolver{T,M}
    qms::QMSolver(T, M)
    N::Int # number of sub-levels (caps by convention here)
end
function NpodSolver(N::Int, args...; kwargs...) where {T, M<:AbstractMatrix{T}}
    adims = DimensionWithSpace(; x0=0, dx=1.0, npnts=N+1, unit="Internal state", periodic=false, spatial=false),

    qms = QMSolver(Matrix{ComplexF64}, adims, args...; wrap=Hermitian, kwargs...)

    NpodSolver(qms, N)
end


# Still need to define accessing methods here.  Maybe what I really want is a standard where these solvers all have qms defined.

"""
    error_checks(nps::NpodSolver)

This will check that the N-pod solver is setup correctly.
"""
function error_check(nps::NpodSolver)
    if dims(nps) != 1
        throw(ArgumentError("N-pod solver must have a single dimension"))
    end
end

"""
    build_rules!(nps::NpodSolver)

This will configure all the rules for the N-pod solver.  We assume that the first
state in the basis is the state which is coupled to the remaining levels.
"""
function build_rules!(nps::NpodSolver)

    add_matrix!(nps, :hamiltonian)

    function coupling(x0, x1; Ωs...)
        Δ = x0[1] - x1[1]
        Ω = Δ > 0 ? Ωs[Δ] : conj(Ωs[-Δ])
        return Ω
    end

    i = 1
    for j in 2:nps.N
        add_rule!(qms, :hamiltonian, ElementRule, coupling, [i,], [j,])
        add_rule!(qms, :hamiltonian, ElementRule, coupling, [j,], [i,])
    end

    set_default_kwargs!(mwrs, :hamiltonian; J=4.0)

end

Base.Meta.ParseError: ParseError:
# Error @ /Users/ispielma/Julia/QMSim.jl/examples/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X20sZmlsZQ==.jl:39:4

end
#  └ ── Expected `end`